In [1]:
import numpy as np      
import matplotlib.pyplot as plt 
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd
import random
from pathlib import Path, PurePath   
from tqdm.notebook import tqdm

## Utility functions

In [2]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio

def plot_spectrogram_and_peaks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """Plots the spectrogram and peaks 

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure()
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()

def load_audio_peaks(audio, offset, duration, hop_size):
    """Load the tracks and peaks of an audio.

    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        offset (float): start reading after this time (in seconds)
        duration (float): only load up to this much audio (in seconds)
        hop_size (int): the hop_length

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, offset=offset, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks
    
    

## Settings

In [21]:
N_TRACKS = 1413 
HOP_SIZE = 512
OFFSET = 1.0
DURATION = 5 
THRESHOLD = 0.7 # the limit to consider a song 'same' to the query one
SIMILARITY_THRESHOLD = 0.2 # the limit to consider a song 'similar' the a query one

In [22]:
data_folder = Path("data/mp3s-32k/")
mp3_tracks = data_folder.glob("*/*/*.mp3")
tracks = data_folder.glob("*/*/*.wav")
query_folder = Path("query_songs/")
query_tracks = query_folder.glob("*.wav")

## Preprocessing

In [23]:
for track in tqdm(mp3_tracks, total=N_TRACKS):
    convert_mp3_to_wav(str(track))

  0%|          | 0/1413 [00:00<?, ?it/s]

## Minhash

In [5]:
def get_audio_signals(num):
    for idx, audio in tqdm(enumerate(tracks)):
        if idx >= num:
            break
        track, sr, onset_env, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE)
        peaks_list.append(peaks) # append the each peak values of a song to peaks_list
        song_name = (str(audio).rsplit('-', 1)[-1]).replace('.wav','') # trim the .wav and folder names from the name of a song
        song_name_list.append(song_name) # append the song name to a list

In [6]:
peaks_list = [] # list of peaks of all the songs
song_name_list = [] # all the songs names
get_audio_signals(N_TRACKS) # list that stores the signals of all songs 

0it [00:00, ?it/s]

In [23]:
# assigning the each unique peak indices to a list
# so that we have a set of peak values of all the signals 
peaks_shingles_list = list(set([item for sublist in peaks_list for item in sublist]))
num_permutation = 200 # number of times we will permute and check if the values are matching
# shuffling the list 'once' because,
# shuffling the list on every permutation and taking the first element is
# equal to shuffling it once and taking the elements from the list within
# the range of number of permutations
random.shuffle(peaks_shingles_list)
song_appear_list = [] # list to store the unique peak values that are appeared on shingles 
for i in range(len(peaks_list)):
    appear_list = [] # a temp list to store each songs peaks 
    for j in range(num_permutation):
        if(peaks_shingles_list[j] in peaks_list[i]):
            # if the peaks match, store it in appear list
            # the 'i'th index of appear_list is also equal to 'i'th index of peaks_list
            # therefore the 'i'th index is also equal to the index of songs names 
            appear_list.append(peaks_shingles_list[j]) 
    # adding all the peaks that appeared of a song to a list
    # each index represents a song and each index is on a parallel order 
    # with the song_name_list
    song_appear_list.append(appear_list)    
            

In [24]:
# getting the jaccard similarity of two lists
def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return round(float(intersection) / union,3)

In [25]:
# find the similarity between query_song's signal and the signals of the songs that we've stored.
def find_similarity_query(song_appear_list, query_song):
    similarity_list = [] # a list to store all the similarity between query and database songs
    for i in range(len(song_appear_list)):
        # find the jaccard similarity of the query song and database songs and store the values in a list 
        similarity = jaccard_similarity(song_appear_list[i], query_song)
        similarity_list.append(similarity)
    return (similarity_list)

In [26]:
def get_query_songs_signals(num):
    # get the query songs signals 
    # same process we do for the songs on database
    for idxx, audio in enumerate(query_tracks):
        if idxx >= num:
            break
        track, sr, onset_env, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE)
        query_peaks_list.append(peaks)
        query_song_name = (str(audio).rsplit('/', 1)[-1]).replace('.wav','')
        query_song_name_list.append(query_song_name)

In [27]:
def find_the_query_song(query_song_name_list, query_peaks_list, song_appear_list, song_name_list):
    print("Similarity \t\t Query track name \t\t Song name")
    for i in range(len(query_song_name_list)):
        match = True
        if(i > 0):
            # if its not the first song, print a line to let us know we're moving to next query song
            print('************************ moving to next song ************************')
        query_song = query_peaks_list[i] # get the query_songs signal
        similarity_list = find_similarity_query(song_appear_list, query_song) # find the similarity
        max_value = max(similarity_list) # get the max value of the similarity
        max_index = similarity_list.index(max_value) # get the index of the max value
        if(similarity_list[max_index] > THRESHOLD): # if the similarity is higher than the 'THRESHOLD' , print it.
            print(similarity_list[max_index],'\t\t\t',query_song_name_list[i],'\t\t\t',song_name_list[max_index])
        else:
            # if its less, than no match! sorry.
            print('oops, no match!')
            match = False # this variable, we'll use to recommend 3 songs
            
        # regardless of match or no match situtation, we ask to user if they want similar songs suggestion.
        choice = input('Do you want to get recommended with similar songs to your query? press y to proceed, any other key to exit')
        if(choice == 'Y' or choice == 'y'): 
            # if the answer is yes, we recommend '3' similar songs
            similarity_list = np.array(similarity_list) # convert to np.array
            # if we have already matched a song, but user still wants similar to query one, then...
            if(match == True):
                # get the 4 max similarity, and delete the first (aka max) one.
                number_of_similar_items = 4
                indices = (-similarity_list).argsort()[:number_of_similar_items]
                indices = np.delete(indices, 0)
            else:
                # if there was no match (aka match == False), then we can take the first 3 as similar.
                number_of_similar_items = 3
                indices = (-similarity_list).argsort()[:number_of_similar_items]
            
            counter = 0 # counter to count, if there is no match higher than the threshold.
            if(len(indices) != 0):
                # if there is a match then...
                for j in range(len(indices)):
                    # get the max index 
                    max_index = indices[j]
                    # if that index's similarity is higher than the threshold, print it, and increase the counter by one.
                    if(similarity_list[max_index] > SIMILARITY_THRESHOLD ):
                        counter+=1
                        print(similarity_list[max_index],'\t\t\t',query_song_name_list[i],'\t\t\t',song_name_list[max_index])
                if(counter < 1):
                    print('oops, no similar songs, what a unique song, hehe!')

            else:
                print('oops, no similar songs, what a unique song, hehe!')



        

In [28]:
query_peaks_list = []
query_song_name_list = []
get_query_songs_signals(10) # '10' stands for the number of query songs
find_the_query_song(query_song_name_list, query_peaks_list, song_appear_list, song_name_list)

Similarity 		 Query track name 		 Song name
0.909 			 track8 			 American_Idiot


Do you want to get recommended with similar songs to your query? press y to proceed, any other key to exit y


0.357 			 track8 			 No_Remorse
0.294 			 track8 			 Erotica
************************ moving to next song ************************
1.0 			 track9 			 Somebody


Do you want to get recommended with similar songs to your query? press y to proceed, any other key to exit o


************************ moving to next song ************************
1.0 			 track10 			 Black_Friday


...

0.267 			 track5 			 Can_t_Go_Back
0.222 			 track5 			 Ireland
0.214 			 track5 			 Salvation
************************ moving to next song ************************
0.909 			 track7 			 Go_Your_Own_Way


Do you want to get recommended with similar songs to your query? press y to proceed, any other key to exit y


0.25 			 track7 			 You_Can_t_Do_That
0.235 			 track7 			 Effigy
0.222 			 track7 			 soul_deep
************************ moving to next song ************************
1.0 			 track6 			 Heartbreaker


Do you want to get recommended with similar songs to your query? press y to proceed, any other key to exit y


0.25 			 track6 			 Nice_Guys_Finish_Last
0.214 			 track6 			 Seconds
